In [43]:
import numpy as np
import pandas as pd
import math

In [2]:
file = pd.read_csv('iris.csv')

In [4]:
data = np.array(file)

In [5]:
#랜덤배열 리스트 저장용
shuffle = []

# 처리한 데이터 길이만큼의 배열 선언
arr = np.arange(len(data))

# 랜덤시드 결정 및 섞기
rs = np.random.RandomState(100)
rs.shuffle(arr)

# 랜덤으로 섞인 인덱스로 새로운 랜덤 데이터 만들기
for i in arr:
    shuffle.append(data[i])

In [65]:
data_len = len(shuffle)

# x, y 데이터 구분
x = []
y = []

for i in range(data_len):
    x.append(shuffle[i][:-1])
    y.append(shuffle[i][-1])
    
sep_len = math.ceil(data_len * 0.7)

# numpy 계산이 필요하므로 numpy 배열로 동시에 바꾼다
tr_x = np.array(x[:sep_len])
ts_x = np.array(x[sep_len:])

tr_y = np.array(y[:sep_len])
ts_y = np.array(y[sep_len:])

In [66]:
def Euclidiean(input_x, tr_x):
    diff_ = tr_x - input_x
    distance = []
    
    for i in range(len(diff_)):
        dis = np.dot(diff_[i],np.matrix.transpose(diff_[i]))
        distance.append(dis)
        
    result = np.array(distance)
    
    return result

In [67]:
def Manhattan(input_x , tr_x):
    diff_ = tr_x - input_x
    distance = []
    
    for i in range(len(diff_)):
        sum_val = np.sum(np.absolute(diff_[i]))
        distance.append(sum_val)
        
    result = np.array(distance)
    
    return result

In [188]:
def Mahalanobis(input_x , tr_x):
    
    diff_ = tr_x - input_x
    distance = []
    
    #만일을 위한 조치
    tr_x = np.array(tr_x , dtype='float') 
    
    cov = np.cov(tr_x.T)
    inv_cov = np.linalg.inv(cov)
    
    for i in range(len(diff_)):      
        cal = np.sqrt(np.dot(diff_[i].T , np.dot(inv_cov,diff_[i])))
        distance.append(cal)
    
    return distance

In [189]:
def KNN(input_x , tr_x, tr_y , k = 3 , dis_way = 'Euc'):
    
    index_list = []
    result_list = []
    dis = []
    
    if(dis_way =='Euc'):
        dis.append(Euclidiean(input_x, tr_x))
    elif(dis_way =='Manh'):
        dis.append(Manhattan(input_x, tr_x))
    elif(dis_way == 'Maha' ):
        dis.append(Mahalanobis(input_x, tr_x))
    else:
        print('Error!')
        return 0
    
    dis = np.array(dis , dtype='float')
    dis = dis.reshape(-1,1)
    
    indexes = np.array(range(len(dis))).reshape(-1,1)
    index_dis = np.concatenate([indexes , dis] , axis=1)
    
    dis_list = sorted( index_dis , key=lambda x:x[1])
    least_dis = dis_list[:k]
    
    for i in least_dis:
        
        least_index = int(i[0])
        result_list.append(tr_y[least_index])
        
    (result ,counts) = np.unique(result_list , return_counts = True)
    max_count = max(counts)
    
    for i in range(len(counts)):
        if(max_count == counts[i]):
            pre = result[i]
    
    return pre

In [195]:
def correction(result, ts_y):
    cor = 0
    length =len(ts_y)
    for i in range(length):
        if(result[i] == ts_y[i]):
            cor += 1
            
    correct = cor/length
            
    return correct

In [201]:
result_list = []
for i in range(len(ts_x)):
    result = KNN(ts_x[i], tr_x,tr_y , dis_way='Maha') 
    result_list.append(result)

In [202]:
correction(result_list,ts_y)

0.8888888888888888